import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
data_dir = "/kaggle/input/animal-image-dataset-90-different-animals"

In [5]:
input_size = (224, 224)
datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)


In [6]:
train_data = datagen.flow_from_directory(data_dir,target_size=input_size,batch_size=32,class_mode="categorical",subset="training")


Found 4320 images belonging to 1 classes.


In [7]:
val_data =datagen.flow_from_directory(data_dir,target_size=input_size,batch_size=32,class_mode="categorical",subset="validation")

Found 1080 images belonging to 1 classes.


In [8]:
augmentation = ImageDataGenerator(rotation_range=20,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode="nearest")


In [9]:
train_augmented = augmentation.flow_from_directory(data_dir,target_size=input_size,batch_size=32,class_mode="categorical",subset="training")

Found 5400 images belonging to 1 classes.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Flatten, Dense

In [11]:
# Initialize the model
model = Sequential()
# Add the firstconvolutional layer
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu',
input_shape=(64, 64, 3)))
# Add the first poolinglayer
model.add(MaxPooling2D(pool_size=(2, 2)))
# Add the second convolutionallayer
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# Add the secondpooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
# Add the flattenlayer
model.add(Flatten())
# Add the first hidden layer
model.add(Dense(units=128,
activation='relu'))
# Add the second hidden layer
model.add(Dense(units=64,
activation='relu'))
# Add the output layer
model.add(Dense(units=90,
activation='softmax'))


In [12]:
model.compile(optimizer='adam',
loss='categorical_crossentropy', metrics=['accuracy'])
# Print the modelsummary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [13]:
test_data =ImageDataGenerator(rescale=1./255)
test_generator = test_data.flow_from_directory('/kaggle/input/animal-image-dataset-90-different-animals',target_size=(64, 64),batch_size=32,class_mode='categorical',shuffle=False)

Found 5400 images belonging to 1 classes.


In [14]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:',test_acc)


169/169 [==============================] - 77s 450ms/step - loss: 405.2198 - accuracy: 0.0000e+00
Test accuracy: 0.0


In [15]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.data import random_split, SubsetRandomSampler
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer
import pytorch_lightning as pl
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image

In [19]:
transform=transforms.Compose([transforms.RandomRotation(10), transforms.RandomHorizontalFlip(),transforms.Resize(224),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])


In [20]:
dataset0=datasets.ImageFolder(root="/kaggle/input/animal-image-dataset-90-different-animals/animals/animals",transform=None)

In [22]:
class_names=dataset0.classes
print(class_names)
print(len(class_names))
class DataModule(pl.LightningDataModule):

    def __init__(self, transform=transform, batch_size=32):
        super().__init__()
        self.root_dir = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"
        self.transform = transform
        self.batch_size = batch_size
    def setup(self, stage=None):
        dataset = datasets.ImageFolder(root=self.root_dir,transform=self.transform)
        n_data = len(dataset)
        n_train = int(0.8 * n_data)
        n_test = n_data - n_train
        train_dataset, test_dataset =torch.utils.data.random_split(dataset, [n_train, n_test])
        self.train_dataset = DataLoader(train_dataset,
        batch_size=self.batch_size, shuffle=True)
        self.test_dataset = DataLoader(test_dataset,batch_size=self.batch_size)
    def train_dataloader(self):
        return self.train_dataset
    def test_dataloader(self):
        return self.test_dataset

['antelope', 'badger', 'bat', 'bear', 'bee', 'beetle', 'bison', 'boar', 'butterfly', 'cat', 'caterpillar', 'chimpanzee', 'cockroach', 'cow', 'coyote', 'crab', 'crow', 'deer', 'dog', 'dolphin', 'donkey', 'dragonfly', 'duck', 'eagle', 'elephant', 'flamingo', 'fly', 'fox', 'goat', 'goldfish', 'goose', 'gorilla', 'grasshopper', 'hamster', 'hare', 'hedgehog', 'hippopotamus', 'hornbill', 'horse', 'hummingbird', 'hyena', 'jellyfish', 'kangaroo', 'koala', 'ladybugs', 'leopard', 'lion', 'lizard', 'lobster', 'mosquito', 'moth', 'mouse', 'octopus', 'okapi', 'orangutan', 'otter', 'owl', 'ox', 'oyster', 'panda', 'parrot', 'pelecaniformes', 'penguin', 'pig', 'pigeon', 'porcupine', 'possum', 'raccoon', 'rat', 'reindeer', 'rhinoceros', 'sandpiper', 'seahorse', 'seal', 'shark', 'sheep', 'snake', 'sparrow', 'squid', 'squirrel', 'starfish', 'swan', 'tiger', 'turkey', 'turtle', 'whale', 'wolf', 'wombat', 'woodpecker', 'zebra']
90


In [23]:
class ConvolutionalNetwork(LightningModule):

    def __init__(self):
        super(ConvolutionalNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(16 * 54 * 54, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)
        self.fc4 = nn.Linear(20, len(class_names))
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 16 * 54 * 54)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.relu(self.fc3(X))
        X = self.fc4(X)
        return F.log_softmax(X, dim=1)
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer
    def training_step(self, train_batch, batch_idx):
        X, y = train_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss
    def validation_step(self, val_batch, batch_idx):
        X, y = val_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("val_loss", loss)
        self.log("val_acc", acc)
    def test_step(self, test_batch, batch_idx):
        X, y = test_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("test_loss", loss)
        self.log("test_acc", acc)

In [25]:
datamodule = DataModule()
datamodule.setup()
train_loader = datamodule.train_dataloader()
for imgs,labels in train_loader:
   break
print(labels)

tensor([ 1, 22, 81, 89, 10, 67, 72,  1,  8, 20, 10, 17, 75, 21,  0, 36, 64,  0,
         3, 67, 44, 37, 14, 41, 86, 86, 37,  4, 26,  3, 75, 87])


In [26]:
if __name__ == '__main__':
 datamodule = DataModule()
 datamodule.setup()
 model = ConvolutionalNetwork()
 trainer = pl.Trainer(max_epochs=10)
 trainer.fit(model, datamodule)
 datamodule.setup(stage='test')
 test_loader = datamodule.test_dataloader()
 trainer.test(dataloaders=test_loader)
    
    

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │   0.024074073880910873    │
│         test_loss         │     4.342234134674072     │
└───────────────────────────┴───────────────────────────┘